# Pose Detection with AlphaPhose

This notebook uses an open source project [MVIG-SJTU/AlphaPose](https://github.com/MVIG-SJTU/AlphaPose) to detect/track multi person poses on a given youtube video.

https://medium.com/the-owl/changing-the-python-version-on-google-colab-245fd510d3aeinstall-python-3-8-kernel-in-google-colaboratory/71511943#71511943

## Install Custom Python version and AlphaPose

In [1]:
!apt-get update
!apt-get install -y build-essential libssl-dev libffi-dev libbz2-dev libncurses5-dev libncursesw5-dev zlib1g-dev libjpeg-dev libpng-dev

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,275 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packag

In [2]:
# Crea ambiente virtuale con Python 3.9
!apt-get install python3.9 python3.9-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!update-alternatives --config python3
!apt-get install python3-pip
!python3 -m pip install --upgrade pip --user

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9-lib2to3 python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9 python3.9-distutils
  python3.9-lib2to3 python3.9-minimal
0 upgraded, 6 newly installed, 0 to remove and 41 not upgraded.
Need to get 5,166 kB of archives.
After this operation, 20.2 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-minimal amd64 3.9.23-1+jammy1 [837 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-minimal amd64 3.9.23-1+jammy1 [2,075 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-stdlib amd64 3.9.23-1+jammy1 [1,842 kB]
Get:4

In [8]:
! python -m pip install "numpy<2.0"
! python -m pip install matplotlib-inline
! python -m pip install git+https://github.com/facebookresearch/visdom.git

  Cloning https://github.com/facebookresearch/visdom.git to /tmp/pip-req-build-h6sdu0ab
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/visdom.git /tmp/pip-req-build-h6sdu0ab
  Resolved https://github.com/facebookresearch/visdom.git to commit 79c0360264eef8b784ea43a34d482efe124411db
  Preparing metadata (setup.py) ... done
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached pillow-11.3.0-cp39-cp39-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached tornado-6.5.2-cp39-abi3-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.8 kB)
  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using c

In [4]:
! python --version

Python 3.9.23


In [ ]:
# ! pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113
! pip3 install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
! pip install scipy
! pip install pillow
! pip install pyyaml
! pip install tqdm matplotlib nibabel pandas opencv-python gdown

In [ ]:
! sudo curl -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
! sudo chmod a+rx /usr/local/bin/yt-dlp

In [ ]:
!python -m pip install cython
!sudo apt-get install libyaml-dev

In [ ]:
import os
! rm -rf /content/AlphaPose
os.chdir('/content/')

In [ ]:
# ! git clone https://github.com/MVIG-SJTU/AlphaPose.git
# os.chdir('/content/AlphaPose')
# print(os.getcwd())
# ! python setup.py build develop

In [ ]:
import os
from os.path import exists, join, basename, splitext

import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q -b pytorch --depth 1 $git_repo_url
  # !cd $project_name && pip install -q -r requirements.txt

import sys
sys.path.append(project_name)

## Download pretrained models

In [ ]:
import gdown
import requests

# Replace 'your-file-id' with the actual file ID from Google Drive
file_id = '1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW'
pretrained_model_path = join(project_name, 'models/sppe/duc_se.pth')
gdown.download(f"https://drive.google.com/uc?id={file_id}", pretrained_model_path, quiet=False)

# yolo_file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
yolo_pretrained_model_path = join(project_name, 'models/yolo/yolov3-spp.weights')
# gdown.download(f"https://drive.google.com/uc?id={file_id}", yolo_pretrained_model_path, quiet=False)

# Make a GET request to download the file
response = requests.get("https://pjreddie.com/media/files/yolov3-spp.weights", stream=True)

# Save the file to your local system
with open(yolo_pretrained_model_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

In [ ]:
## fix code for AlphaPose

with open("/content/AlphaPose/fn.py", "r") as f:
    python_file = "".join(f.readlines())

python_file = python_file.replace("from torch._six import string_classes, int_classes", "from torch._six import string_classes\nint_classes = int")

with open("/content/AlphaPose/fn.py", "w") as f:
    f.write(python_file)

## Detect poses on a test video

We are going to detect poses on the following youtube video:

In [ ]:
YOUTUBE_ID = '0vnMdFmZ3P8'

YouTubeVideo(YOUTUBE_ID)

Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [ ]:
# ! rm -df youtube.mp4
# download the youtube with the given ID
! yt-dlp --output "youtube.mp4" https://www.youtube.com/watch?v=$YOUTUBE_ID
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4

In [ ]:
# run AlphaPose_video on these 5 seconds video
# !rm -rf AlphaPose_video.avi

!cd AlphaPose && python3 video_demo.py --sp --video ../video.mp4 --outdir .. --save_video --vis_fast
#! python3 AlphaPose/video_demo.py --sp --video video.mp4 --outdir . --save_video

# convert the result into MP4
!ffmpeg -y -loglevel info -i AlphaPose_video.avi AlphaPose_video.mp4

Finally, visualize the result:

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('AlphaPose_video.mp4', width=960, height=720)